In [0]:
# configs = {"fs.azure.account.key.storage001de.blob.core.windows.net": "MA38y0ZA2Ij1L3QXl/F/Y8EGXR1VjTGb896PNbUmPT99uhVLoi0qCEiBEBagBQ4yscqwq9Zxb0vX+AStlr0Z3Q=="}

# # create mount to gold
# dbutils.fs.mount(
# 	source = "wasbs://gold@storage001de.blob.core.windows.net/",
# 	mount_point = "/mnt/datalake/gold",
# 	extra_configs = configs
# )
# # Đường dẫn đến mount point cần gỡ bỏ
# # mount_point = "/mnt/datalake/gold/"

# # # Gỡ bỏ mount point
# # dbutils.fs.unmount(mount_point)


In [0]:
# %fs ls /mnt/datalake/gold


In [0]:
from pyspark.sql.functions import *

# Define the gold location
fact_goldLocation = "/mnt/datalake/gold/fact_movie"

# Table Name
silver_movie = "silver.movie"
factMoive = "gold.Fact_Movie"

# Dim Table
dimInfo = "gold.Dim_Info"
dimDate = "gold.Dim_Date"
dimLanguage = "gold.Dim_Language"
dimCompany = "gold.Dim_Company"
dimMovieCompany = "gold.Dim_Movie_Company"
dimGenre = "gold.Dim_Genre"
dimMovieGenre = "gold.Dim_Movie_Genre"

In [0]:
spark.sql(f"DROP TABLE IF EXISTS {factMoive}")


DataFrame[]

In [0]:
# Create schema
spark.sql("CREATE SCHEMA IF NOT EXISTS gold")
spark.sql(f"""
            CREATE EXTERNAL TABLE IF NOT EXISTS {factMoive} (
                movieId int,
                dateId int,
                languageId int,
                popularity float,
                budget int,
                revenue int,
                rating float,
                voteCount int)
            USING delta
            LOCATION '{fact_goldLocation}'
        """)

DataFrame[]

In [0]:
dataSilver = spark.read.table(silver_movie).select('id', 'release', 'language', 'popularity', 'budget', 'revenue', 'rating', 'voteCount')
dataSilver.show()

+---+----------+--------+----------+--------+--------+------+---------+
| id|   release|language|popularity|  budget| revenue|rating|voteCount|
+---+----------+--------+----------+--------+--------+------+---------+
|  1|2002-02-14|      en|  7.196356|48000000|76019048|   6.4|    87814|
|  2|1960-01-01|      xx|         0|       0|       0|   6.7|      338|
|  3|1960-01-11|      fr|  9.935891|       0|   60254|   7.6|    34150|
|  4|1960-05-13|      fr|  8.200586|  400000|  594902|   7.7|    86132|
|  5|1960-07-01|      en|  6.586372|       0|       0|   6.0|     6851|
|  6|1961-07-14|      en|   4.73816|12000000| 7900000|   6.8|    17070|
|  7|1960-10-21|      en| 11.994281| 3000000|18778738|   8.3|   191097|
|  8|1961-12-14|      it|  9.595757|       0|    2606|   7.7|    31909|
|  9|1960-11-04|      en|  4.605205| 2800000|       0|   6.3|     7149|
| 10|1961-03-17|      en|  0.803604| 6000000|       0|   6.3|     2443|
| 11|1961-01-22|      en|  2.000936| 5421000|       0|   6.4|   

In [0]:
dataDimInfo = spark.read.table(dimInfo)
dataDimDate = spark.read.table(dimDate)
dataDimLanguage = spark.read.table(dimLanguage)
dataDimCompany = spark.read.table(dimCompany)
dataDimMovieCompany = spark.read.table(dimMovieCompany)
dataDimGenre = spark.read.table(dimGenre)
dataDimMovieGenre = spark.read.table(dimMovieGenre)

In [0]:
# mappedData = dataSilver \
#         .join(dataDimDate, dataSilver.id = dataDimDate.dateId, how="left")

In [0]:
dataFact = dataSilver \
    .join(dataDimDate, (year(dataSilver['release']) == dataDimDate['year']) & (month(dataSilver['release']) == dataDimDate['month']), how = 'left') \
    .join(dataDimLanguage, dataSilver['language'] == dataDimLanguage['lang_code'], 'left') \
    .select(dataSilver['id'], dataDimDate['id'].alias('dateId'), dataDimLanguage['id'].alias('languageId'), 'popularity', 'budget', 'revenue', 'rating', 'voteCount')

# Đổi tên cột 'id' ở vị trí cụ thể (ví dụ: cột thứ hai)
# column_index = 1  # Chỉ mục cột (tính từ 0)
# renamed_df = df.withColumnRenamed(df.columns[column_index], 'new_id')  # Đổi tên thành 'new_id' hoặc tên mới tùy ý

dataFact.show()


+---+------+----------+----------+--------+--------+------+---------+
| id|dateId|languageId|popularity|  budget| revenue|rating|voteCount|
+---+------+----------+----------+--------+--------+------+---------+
|  1|  1226|         1|  7.196356|48000000|76019048|   6.4|    87814|
|  2|   721|        42|         0|       0|       0|   6.7|      338|
|  3|   721|        58|  9.935891|       0|   60254|   7.6|    34150|
|  4|   725|        58|  8.200586|  400000|  594902|   7.7|    86132|
|  5|   727|         1|  6.586372|       0|       0|   6.0|     6851|
|  6|   739|         1|   4.73816|12000000| 7900000|   6.8|    17070|
|  7|   730|         1| 11.994281| 3000000|18778738|   8.3|   191097|
|  8|   744|        31|  9.595757|       0|    2606|   7.7|    31909|
|  9|   731|         1|  4.605205| 2800000|       0|   6.3|     7149|
| 10|   735|         1|  0.803604| 6000000|       0|   6.3|     2443|
| 11|   733|         1|  2.000936| 5421000|       0|   6.4|     3307|
| 12|   726|        

#### Merge into Fact table

In [0]:
 
from delta.tables import *

#check if the bronze location contain the delta table
dataFact.write.mode("overwrite").format("delta").option("overwriteSchema", "true").save(fact_goldLocation)

In [0]:
# Maintenance for Delta table
dataDelta = DeltaTable.forName(spark, factMoive)

if dataDelta.history(30).filter("operation = 'VACUUM START'").count() == 0:
	dataDelta.optimize()
	dataDelta.vacuum()


In [0]:
%sql
select * from gold.fact_movie

id,dateId,languageId,popularity,budget,revenue,rating,voteCount
1,1226,1,7.196356,48000000,76019048,6.4,87814
2,721,42,0,0,0,6.7,338
3,721,58,9.935891,0,60254,7.6,34150
4,725,58,8.200586,400000,594902,7.7,86132
5,727,1,6.586372,0,0,6.0,6851
6,739,1,4.73816,12000000,7900000,6.8,17070
7,730,1,11.994281,3000000,18778738,8.3,191097
8,744,31,9.595757,0,2606,7.7,31909
9,731,1,4.605205,2800000,0,6.3,7149
10,735,1,0.803604,6000000,0,6.3,2443
